In [ ]:
from langfuse import get_client
from dotenv import load_dotenv

load_dotenv()
langfuse = get_client()

In [ ]:
from tqdm import tqdm

# List dataset files, open each JSON file (each contains a list of dicts), and iterate items
import os
import json

# 1. Directory containing the dataset JSON files (relative to repo root / notebook)
dataset_dir = os.path.join("datasets")

# 1. Create a list containing all file names from datasets folder
dataset_files = sorted([fn for fn in os.listdir(dataset_dir) if fn.endswith('.json')])
print(f"Found {len(dataset_files)} dataset files:")
for fn in dataset_files:
    print(f" - {fn}")

# 2. Open and read each file. Each file is expected to contain a list of dicts.
all_data = {}
for fname in dataset_files:
    path = os.path.join(dataset_dir, fname)
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    if not isinstance(data, list):
        print(f"Warning: file {fname} does not contain a list (got {type(data)}). Skipping.")
        continue

    all_data[fname] = data

    # 3. Iterate over dict items
    print(f"\nFile: {fname} contains {len(data)} items. Showing up to first 3 items:")
    for i, item in enumerate(data[:3]):
        if isinstance(item, dict):
            keys = list(item.keys())
            # Short preview of values (truncate long strings)
            preview = json.dumps(item, ensure_ascii=False)
            if len(preview) > 300:
                preview = preview[:300] + ' ...[truncated]'
            print(f"  Item {i}: keys={keys}")
            print(f"    preview: {preview}")
        else:
            print(f"  Item {i}: not a dict (type={type(item)}) -> {item}")

print("\nAll files loaded into variable 'all_data' (mapping filename -> list of items).")

In [ ]:
from data_extraction.adapters.albert_structured_extractor import AlbertStructuredExtractor
from pydantic import BaseModel, Field
from tqdm import tqdm

class Categorie(BaseModel):
    categorie : str = Field(title="Catégorie de classification")

llm = AlbertStructuredExtractor(Categorie)

# Get production prompt
PROMPT = langfuse.get_prompt("Classification", label="latest")
file_name = "hierarchical_slice1.json"

accuracy = 0
fn_autre = 0
preds = []
labels = []
for chunk in tqdm(all_data[file_name]):
    titre = chunk["input"]["title"]
    current_chunk = chunk["input"]["chunk"]
    next_chunks = chunk["input"]["next_chunks"]
    previous_chunks = chunk["input"]["previous_chunks"]
    full_prompt = PROMPT.compile(
        title=titre,
        previous_chunks=previous_chunks,
        chunk=current_chunk,
        next_chunks=next_chunks
    )
    label = chunk["expected_output"]
    answer = llm.get_structured_output("albert-small", system_prompt=full_prompt[0]["content"], user_message=full_prompt[1]["content"], temperature=0.2)
    json_answer = answer.get_json()
    preds.append(json_answer["categorie"])
    labels.append(label)
    if label == "autre" and json_answer["categorie"] != "autre":
        fn_autre += 1
    if json_answer["categorie"] == label:
        accuracy += 1
print(f"Accuracy: {accuracy/len(all_data[file_name])*100:.2f}%")
print(f"False negatives (autre): {fn_autre/len(all_data[file_name])*100:.2f}%")

In [ ]:
print(f"False negatives (autre): {fn_autre/len(all_data[file_name])*100:.2f}%")

In [ ]:
full_prompt = PROMPT.compile(
        title="titre",
        previous_chunks="previous_chunks",
        chunk="current_chunk",
        next_chunks="next_chunks"
    )
full_prompt[0]["content"]

In [ ]:
PROMPT = langfuse.get_prompt("Classification", label="latest")
test_dict = {
    "chunk": "chunk",
    "previous_chunks": "prev_cunk",
    "next_chunks": "next_chunk",
    "title": "titre"
}
compiled_prompt = PROMPT.compile(**test_dict)

In [ ]:
from data_extraction.adapters.hierarchical_file_chunker import HierarchicalFileChunker

all_chunks = []
chunker = HierarchicalFileChunker(separators=["##### ", "#### ", "### ", "## ", "#"])
for file_path in ["sample_pdf/agence_eau.pdf", "sample_pdf/reinsertion_professionnelle.pdf", "sample_pdf/relance_exploitation.pdf"]:
    chunks = chunker.chunk_file(file_path)
    all_chunks.extend(chunks)

In [ ]:
import json

eval_chunks = []
slice_size = 5
no_chunk_msg = "No chunk is available."

with open("datasets/base_parser.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for i, chunk_obj in enumerate(all_chunks):
    # Extract text content for easier handling
    chunk_text = chunk_obj.get("content", "")

    # Previous chunks
    if i == 0:
        prev_chunks = no_chunk_msg
    else:
        prev_chunks = [all_chunks[j]["content"] for j in range(max(0, i - slice_size), i)]

    # Next chunks
    if i >= len(all_chunks) - 1:
        next_chunks = no_chunk_msg
    else:
        next_chunks = [all_chunks[j]["content"] for j in range(i + 1, min(i + 1 + slice_size, len(all_chunks)))]

    eval_chunks.append(
        {
            "input": 
            {
                "chunk": chunk_text,
                "previous_chunks": "\n\n".join(prev_chunks) if isinstance(prev_chunks, list) else prev_chunks,
                "next_chunks": "\n\n".join(next_chunks) if isinstance(next_chunks, list) else next_chunks,
                "title": chunk_obj["title"]
            },
            "expected_output": data[i]["expected_output"]
        }
    )

# 3. Réécrire la liste mise à jour dans le fichier
with open("fruits.json", "w", encoding="utf-8") as f:
    json.dump(eval_chunks, f, ensure_ascii=False, indent=4)

print("Items ajoutés avec succès !")

In [ ]:
eval_chunks

In [ ]:
from langfuse.openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("ALBERT_API_KEY"),
    base_url="https://albert.api.etalab.gouv.fr/v1"
)

PROMPT = langfuse.get_prompt("Classification", label="latest")

# Define your task function
def my_task(*, item, **kwargs):
    context = item["input"]
    compiled_prompt = PROMPT.compile(**context)
    client = OpenAI(
        api_key=os.getenv("ALBERT_API_KEY"),
        base_url="https://albert.api.etalab.gouv.fr/v1"
    )

    print(compiled_prompt)
    response = client.chat.completions.create(
        model="albert-large", 
        messages=compiled_prompt
    )
 
    return {"output": response.choices[0].message.content}

# Run experiment on local data
local_data = [
    {
        "input": 
            {
                "chunk": "Il.1. Nature et durée de l'activité du demandeur\n\nPour bénéficier du dispositif de l'aide à la réinsertion professionnelle, le demandeur doit justifier à la date de dépôt du dossier de 5 années d'activité agricole au sens de l'art. L. 311-1 du code rural et de la pêche maritime! (sont cependant exclues les activités aquacoles et équestres), précédant-mmédiatementde-dépôt dea-demande-d'ARP. en qualité de :\n\n- ° exploitant agricole ou associé exploitant, à titre principal, affilié à l'assurance maladie, invalidité, maternité des personnes non-salariées des professions agricoles (AMEXA), ou\n- ° conjoint de chef d'exploitation à titre principal participant aux travaux ou de conjoint collaborateur, bénéficiant à ce titre de l'AMEXA, ou\n- ° aide familial bénéficiant de l'AMEXA.",
                "previous_chunks": "## IL Conditions d'éligibilité du demandeur",
                "next_chunks": "## 11.2. Engagements du demandeur\n\nLe bénéficiaire de l'aide à la réinsertion professionnelle :\n\n- ° doit s'engager à ne pas revenir à l'agriculture en qualité de chef d'exploitation ou d'entreprise agricole, de conjoint ou d'aide ( cf. 11.1) pendant une durée de 5 ans à compter de l'attribution de l'aide (date de la décision préfectorale d'octroi de l'aide) ;\n- ° peut toutefois conserver une parcelle de subsistance qui ne doit pas excéder un hectare de surface agricole utile pondérée (SAUP) ;\n- e ne doit pas être à deux ans de l'âge légal de la retraite, ou à la retraite à la date de dépôt du dossier.\n\n1 L'artide L. 311-1 du code rural dispose que : « Sont réputées agricoles toutes les activités correspondant à la maîtrise d'un cycle biologique de caractère végétal ou animal et constituant une ou plusieurs étapes nécessaires au déroulement de ce cycle ainsi que les activités exercées par un exploitant agricole qui sont dans le prolongement de l'acte de production ou qui ont pour support l'exploitation. Les activités marines sont réputées agricoles, nonobstant le statut social dont relèvent ceux qui les pratiquent. Il en est de même des activités de préparation et d'entraînement des équidés domestiques en vue de leur exploitation, à l'exclusion des activités de spectacle.",
                "title": "IL Conditions d'éligibilité du demandeur"
            },
        "expected_output": "eligibilite"
    },
]

result = langfuse.run_experiment(
    name="Geography Quiz",
    description="Testing basic functionality",
    data=local_data,
    task=my_task,
)

print(result.format())

# Run once

In [ ]:
# Display chunks with their titles to show document structure
print(f"Total chunks: {len(new_chunks)}\n")
print("=" * 80)

for i, chunk in enumerate(new_chunks[:10], 1):  # Show first 10 chunks
    title = chunk.get('title', 'No title')
    content_preview = chunk['content'][:100].replace('\n', ' ')
    
    print(f"\nChunk {i}")
    print(f"Title: {title}")
    print(f"Content preview: {content_preview}...")
    print("-" * 80)

In [ ]:
client = get_client()

# Chunk Classification with LLM

Classification of chunks into predefined categories based on the dispositif d'aide schema.

In [ ]:
from typing import Type, List, Optional
from pydantic import BaseModel, Field
from enum import Enum


# Define classification categories based on schema_dispositif_aide
class ChunkCategory(str, Enum):
    """Categories for chunk classification based on dispositif aide schema"""
    PRESENTATION_AIDE = "presentation_aide"  # Title and description
    ELIGIBILITE = "eligibilite"  # Who can access, eligible projects
    TYPE_AIDE = "type_aide"  # Types of financial aid
    PORTEURS = "porteurs"  # Actors involved in implementation
    INFORMATIONS_EXTERNES = "informations_externes"  # External links and parent programs
    BENEFICIAIRES = "beneficiaires"  # Target beneficiaries
    ELIGIBILITE_GEOGRAPHIQUE = "eligibilite_geographique"  # Geographic coverage
    DATES = "dates"  # Opening and closing dates
    OPERATIONS_ELIGIBLES = "operations_eligibles"  # Eligible operations/expenses
    CADRE_LEGAL = "cadre_legal"  # Legal framework and regulations
    AUTRE = "autre"  # Other/Uncategorized


class ClassifiedChunk(BaseModel):
    """Schema for a classified chunk"""
    chunk_id: str = Field(..., description="ID of the chunk")
    content: str = Field(..., description="Content of the chunk")
    category: ChunkCategory = Field(..., description="Classified category")
    confidence: float = Field(..., description="Confidence score (0-1)", ge=0, le=1)
    reasoning: Optional[str] = Field(None, description="Explanation for the classification")


class ChunkClassificationResult(BaseModel):
    """Schema for classification results"""
    classifications: List[ClassifiedChunk] = Field(..., description="List of classified chunks")

In [ ]:
# Self-test: synthetic labels (small), build and export matrix to local files (in notebook folder)
true = ['eligibilite','presentation_aide','operations_eligibles','eligibilite','autre','operations_eligibles','autre']
                
pred = ['eligibilite','presentation_aide','autre','eligibilite','autre','operations_eligibles','presentation_aide']
labels = ['presentation_aide','eligibilite','operations_eligibles','autre']
cm = build_confusion_matrix(true, pred, labels=labels)
print('Confusion matrix (counts):')
print(cm)

# Export CSV and PNG (png will be shown inline as well)
export_confusion_matrix(cm, path_csv='confusion_matrix_counts.csv', path_png='confusion_matrix_heatmap.png', normalize=True)
print('Exported files: confusion_matrix_counts.csv, confusion_matrix_heatmap.png')